Making a LeNet 5 CNN Model on Fashion MNIST Dataset using Pytorch

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torch import optim
from torchsummary import summary



In [2]:
training=datasets.CIFAR100(root="./data",train=True,download=True,transform=transforms.Compose([transforms.Resize((227,227)),transforms.ToTensor()]))
testing=datasets.CIFAR100(root="./data",train=False,download=True,transform=transforms.Compose([transforms.Resize((227,227)),transforms.ToTensor()]))

Files already downloaded and verified
Files already downloaded and verified


In [3]:
train_dataloader=DataLoader(training,batch_size=100)
test_dataloader=DataLoader(testing,batch_size=100)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.c1=nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4,padding=0)
        self.s1=nn.MaxPool2d(kernel_size=3,stride=2)
        self.c2=nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,stride=1,padding=2)
        self.s2=nn.MaxPool2d(kernel_size=3,stride=2)
        self.c3=nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1)
        self.c4=nn.Conv2d(in_channels=384,out_channels=384,kernel_size=3,stride=1,padding=1)
        self.c5=nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,stride=1,padding=1)
        self.s3=nn.MaxPool2d(kernel_size=3,stride=2)
        self.flatten= nn.Flatten(start_dim=1)
        self.l1=nn.Linear(in_features=256*6*6,out_features=4096)
        self.l2=nn.Linear(in_features=4096,out_features=4096)
        self.l3=nn.Linear(in_features=4096,out_features=1000)
        self.act=nn.ReLU()
        self.d1=nn.Dropout()
        self.d2=nn.Dropout()

    def forward(self,x):
        x=self.c1(x)
        x=self.act(x)
        x=self.s1(x)
        x=self.c2(x)
        x=self.act(x)
        x=self.s2(x)
        x=self.c3(x)
        x=self.act(x)
        x=self.c4(x)
        x=self.act(x)
        x=self.c5(x)
        x=self.act(x)
        x=self.s3(x)
        x=self.d1(x)
        x=self.flatten(x)
        x=self.l1(x)
        x=self.act(x)
        x=self.d2(x)
        x=self.l2(x)
        x=self.act(x)
        x=self.l3(x)
        return x
        
    



In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model=NeuralNetwork().to(device)
learning_rate=0.001


In [7]:
loss_fun=nn.CrossEntropyLoss().to(device)
optimiser=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [8]:
# summary(model,input_size=(3,227,227))

In [9]:
print(device)

cuda


In [12]:
from tqdm.notebook import tqdm
train_dataloader_1=DataLoader(training,batch_size=1)
data, target = next(iter(train_dataloader_1))

for i in tqdm(range(1000)):
    model.train()
    data=data.to(device)
    target=target.to(device)
    scores=model(data)
    loss=loss_fun(scores,target)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    # print(sc)
        
        



  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from tqdm.notebook import tqdm
import tensorboard as tb
for i in tqdm(range(10)):
    for i,(data,target) in tqdm(enumerate(train_dataloader)):
        model.train()
        data=data.to(device)
        target=target.to(device)
        scores=model(data)
        loss=loss_fun(scores,target)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        print(loss)
        
        



In [11]:
def accuracy_check(loader,model):
    correct=0
    total=0
    model.eval()
    with torch.no_grad():
        for (x,y) in tqdm(loader):
            x=x.to(device=device)
            y=y.to(device=device)
            scores=model(x)
            scores=nn.functional.softmax(scores,dim=1)
            _,predictions=scores.max(1)
            correct+=(predictions==y).sum()
            total+= predictions.size(0)
    print(float(correct/total))

In [12]:
model.eval()
accuracy_check(test_dataloader,model)

  0%|          | 0/100 [00:00<?, ?it/s]

0.1889999955892563
